# 전처리 2

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
pd.options.mode.chained_assignment = None  # 경고 숨기기

# 1~4. 기존과 동일 --------------------- </br>
    (1) Date 컬럼 형식 맞추기 - '실제 주식 데이터'와 비교 위해서 </br>
    (2) 텍스트 정제된 ppc_Title 컬럼 만들기</br>
    (3) ppc_Title 컬럼 중복 확인 및 제거</br>
    (4) 띄어쓰기 기준 명사만 추출하여 Tokens 컬럼 만들기

In [2]:
### 4. 띄어쓰기 후 각 단어 형태소 분석 -> 명사만 추출
kiwi = Kiwi(typos='basic')
stopwords = Stopwords()

def tokenize_text(text):
    # words = kiwi.space(text)  # (1) 띄어 쓰기 교정      ##교수님 거에 있어서 추가했어!!!!!!!
    words = text.split()  # (2) 띄어쓰기로 텍스트 나누기

    token_list = []
    for word in words:
        tokens = kiwi.tokenize(word)  # (3) 각 단어를 형태소 분석
        for token in tokens:
            if token.tag in ['NNG', 'NNP']:   # (4) 명사(NNG, NNP)만 추출
                token_list.append(token.form)
    
    # token_list = ' '.join(token_list) # (5) 모든 문장을 하나의 string으로 join    ##교수님 거에 있어서 추가했어!!!!!!! Token 컬럼에  [ ] 없애주는 거야!
    return token_list

In [3]:
# 데이터 불러오기
news = pd.read_csv('naver_01.Preprocessing_FIN.csv')
news.head()

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,네이버 금지 혐오 표현 기준 개정 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,네이버 거대 상공인 라이브 커머스 대본 작성 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,네이버 악성 댓글 때 악플 꼬리표
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버 중소 상공인 브랜드 구축
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,네이버 카카오 인물 정보 장례 지도사 직업 추가


# <span style="color:pink"> ◤ 전처리 과정 추가 ---------------------

# 5. 고유명사 직접 추가하여 Tokens 수정하기 

In [4]:
# 사용자 정의 사전에 추가할 단어 목록
N_words = ['네이버', '네이버페이', '네이버 페이', '네이버웹툰', '네이버 웹툰', '네이버클라우드', '네이버 클라우드',
            '네이버쇼핑', '네이버 쇼핑', '네이버뉴스', '네이버 뉴스', '네이버스토어', '네이버 스토어',
            '네이버 하이퍼클로바', '네이버 하이퍼클로바X', '네이버 하이퍼클로바 X',
            '하이퍼 클로바 X','하이퍼클로바 X','하이퍼클로바','하이퍼 클로바','클로바 X','클로바',
            '데이터 센터', '데이터센터','데이터'
            '네이버라인', '네이버 라인', '라인', '야후', '라인 야후', '라인야후',
             '카카오', '카카오톡', '카카오페이', '카카오 페이', '쿠팡', '트위터', '삼성전자']

n_words =['인공지능', '인공 지능', 'AI', '생성 AI','생성AI', '생성형 AI', '생성형AI',
           '앱', '애플리케이션']

# NNP는 고유명사, NNG는 일반명사
for word in N_words:
    kiwi.add_user_word(word, "NNP")
for word in N_words:
    kiwi.add_user_word(word, "NNG")

In [5]:
news['Tokens'] = news['ppc_Title'].apply(tokenize_text)

In [6]:
news.head(20)
# # 8 => 네이버 투자 패션 플랫폼 브랜디 자금 수혈 필요
# # 17 => 네이버페이 대출 오픈 금리 포인트

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,"[네이버, 금지, 혐오, 표현, 기준, 개정, 정책, 적용]"
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,"[네이버, 거대, 상공인, 라이브, 커머스, 대본, 작성, 이유]"
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,"[네이버, 악성, 댓글, 때, 악플, 꼬리표]"
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,"[네이버, 중소, 상공인, 브랜드, 구축]"
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"[네이버, 카카오, 인물, 정보, 장례, 지도사, 직업, 추가]"
5,2023-06-01,"현대차證 “네이버, 하이퍼클로바X 시장 연착륙할 것”",현대차證 네이버 하이퍼클로바X 시장 연착륙할 것,"[현대차, 네이버, 하이퍼 클로바 X, 시장, 연착륙]"
6,2023-06-01,"재난상황 ‘주연’된 네이버...공공인프라 된 플랫폼, 책임 어디까지?",재난상황 주연 된 네이버 공공인프라 된 플랫폼 책임 어디까지,"[재난, 상황, 주연, 네이버, 공공, 인프라, 플랫폼, 책임]"
7,2023-06-01,"AI 웹툰 논란에 네이버·카카오 ""인간 작품만 받는다""",AI 웹툰 논란에 네이버 카카오 인간 작품만 받는다,"[웹툰, 논란, 네이버, 카카오, 인간, 작품]"
8,2023-06-01,네이버가 300억 투자한 패션 플랫폼 ‘브랜디’ 자금 수혈 필요,네이버가 300억 투자한 패션 플랫폼 브랜디 자금 수혈 필요,"[네이버, 투자, 패션, 플랫폼, 브랜디, 자금, 수혈, 필요]"
9,2023-06-01,"네이버 마비, 5분 만에 복구됐지만… 실제 재난에선 어쩌나",네이버 마비 5분 만에 복구됐지만 실제 재난에선 어쩌나,"[네이버, 마비, 복구, 재난]"


# 6. 동의어 하나로 매핑 tokens 수정

In [7]:
# 동의어 매핑
replace_map = {
    '네이버 페이': '네이버페이',
    '네이버 웹툰': '네이버웹툰',
    '네이버 클라우드': '네이버클라우드',
    '네이버 쇼핑': '네이버쇼핑',
    '네이버 뉴스': '네이버뉴스',
    '네이버 스토어': '네이버스토어',

    '네이버라인': '라인야후',
    '네이버 라인': '라인야후',
    '야후': '라인야후',
    '라인 야후':'라인야후',

    '네이버 하이퍼클로바': '하이퍼클로바X',
    '네이버 하이퍼클로바X': '하이퍼클로바X',
    '네이버 하이퍼클로바 X': '하이퍼클로바X',
    '하이퍼 클로바 X': '하이퍼클로바X',
    '하이퍼클로바 X': '하이퍼클로바X',
    '하이퍼클로바': '하이퍼클로바X',
    '하이퍼 클로바': '하이퍼클로바X',
    '클로바 X': '하이퍼클로바X',
    '클로바': '하이퍼클로바X',
    '데이터 센터': '데이터센터',

    '카카오톡' : '카카오',
    '카오' : '카카오',
    '카카오 페이': '카카오페이',
    '삼성 전자': '삼성전자',
    '인공 지능': 'AI',
    '인공지능': 'AI',
    '생성 AI' : '생성형AI',
    '생성AI': '생성형AI',
    '생성형 AI': '생성형AI'
}

In [8]:
# 동의어 매핑을 tokens에 적용하여 수정
news['Tokens'] = news['Tokens'].apply(lambda x: [replace_map.get(item, item) for item in x])

In [9]:
news.head(20)

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,"[네이버, 금지, 혐오, 표현, 기준, 개정, 정책, 적용]"
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,"[네이버, 거대, 상공인, 라이브, 커머스, 대본, 작성, 이유]"
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,"[네이버, 악성, 댓글, 때, 악플, 꼬리표]"
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,"[네이버, 중소, 상공인, 브랜드, 구축]"
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"[네이버, 카카오, 인물, 정보, 장례, 지도사, 직업, 추가]"
5,2023-06-01,"현대차證 “네이버, 하이퍼클로바X 시장 연착륙할 것”",현대차證 네이버 하이퍼클로바X 시장 연착륙할 것,"[현대차, 네이버, 하이퍼클로바X, 시장, 연착륙]"
6,2023-06-01,"재난상황 ‘주연’된 네이버...공공인프라 된 플랫폼, 책임 어디까지?",재난상황 주연 된 네이버 공공인프라 된 플랫폼 책임 어디까지,"[재난, 상황, 주연, 네이버, 공공, 인프라, 플랫폼, 책임]"
7,2023-06-01,"AI 웹툰 논란에 네이버·카카오 ""인간 작품만 받는다""",AI 웹툰 논란에 네이버 카카오 인간 작품만 받는다,"[웹툰, 논란, 네이버, 카카오, 인간, 작품]"
8,2023-06-01,네이버가 300억 투자한 패션 플랫폼 ‘브랜디’ 자금 수혈 필요,네이버가 300억 투자한 패션 플랫폼 브랜디 자금 수혈 필요,"[네이버, 투자, 패션, 플랫폼, 브랜디, 자금, 수혈, 필요]"
9,2023-06-01,"네이버 마비, 5분 만에 복구됐지만… 실제 재난에선 어쩌나",네이버 마비 5분 만에 복구됐지만 실제 재난에선 어쩌나,"[네이버, 마비, 복구, 재난]"


# 7. 불용어 리스트와 제거 함수 만들어서 tokens 수정

In [10]:
# 불용어 제거 함수 
stopwords = Stopwords()
stopwords_list = [st[0] for st in stopwords.stopwords]
stopwords_list.append('네이버')

def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords_list]

In [11]:
# 불용어 제거 함수 적용
news_stop = news.copy()
news_stop['Stop_tokens'] = news_stop['Tokens'].apply(remove_stopwords)
news_stop.head(20)

,Date,Title,ppc_Title,Tokens,Stop_tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,"[네이버, 금지, 혐오, 표현, 기준, 개정, 정책, 적용]","[금지, 혐오, 표현, 기준, 개정, 정책, 적용]"
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,"[네이버, 거대, 상공인, 라이브, 커머스, 대본, 작성, 이유]","[거대, 상공인, 라이브, 커머스, 대본, 작성, 이유]"
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,"[네이버, 악성, 댓글, 때, 악플, 꼬리표]","[악성, 댓글, 악플, 꼬리표]"
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,"[네이버, 중소, 상공인, 브랜드, 구축]","[중소, 상공인, 브랜드, 구축]"
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"[네이버, 카카오, 인물, 정보, 장례, 지도사, 직업, 추가]","[카카오, 인물, 정보, 장례, 지도사, 직업, 추가]"
5,2023-06-01,"현대차證 “네이버, 하이퍼클로바X 시장 연착륙할 것”",현대차證 네이버 하이퍼클로바X 시장 연착륙할 것,"[현대차, 네이버, 하이퍼클로바X, 시장, 연착륙]","[현대차, 하이퍼클로바X, 시장, 연착륙]"
6,2023-06-01,"재난상황 ‘주연’된 네이버...공공인프라 된 플랫폼, 책임 어디까지?",재난상황 주연 된 네이버 공공인프라 된 플랫폼 책임 어디까지,"[재난, 상황, 주연, 네이버, 공공, 인프라, 플랫폼, 책임]","[재난, 상황, 주연, 공공, 인프라, 플랫폼, 책임]"
7,2023-06-01,"AI 웹툰 논란에 네이버·카카오 ""인간 작품만 받는다""",AI 웹툰 논란에 네이버 카카오 인간 작품만 받는다,"[웹툰, 논란, 네이버, 카카오, 인간, 작품]","[웹툰, 논란, 카카오, 인간, 작품]"
8,2023-06-01,네이버가 300억 투자한 패션 플랫폼 ‘브랜디’ 자금 수혈 필요,네이버가 300억 투자한 패션 플랫폼 브랜디 자금 수혈 필요,"[네이버, 투자, 패션, 플랫폼, 브랜디, 자금, 수혈, 필요]","[투자, 패션, 플랫폼, 브랜디, 자금, 수혈, 필요]"
9,2023-06-01,"네이버 마비, 5분 만에 복구됐지만… 실제 재난에선 어쩌나",네이버 마비 5분 만에 복구됐지만 실제 재난에선 어쩌나,"[네이버, 마비, 복구, 재난]","[마비, 복구, 재난]"


# 8. 각 행의 리스트를 문자열로 결합하고 컴마 제거


In [12]:
news_stop_copy = news_stop.copy()

In [13]:
# Tokens 열을 문자열로 변환
news_stop_copy['Stop_tokens'] = news_stop_copy['Stop_tokens'].astype(str)

In [14]:
# Tokens 열에서 리스트 기호와 쉼표 제거
news_stop_copy['Stop_tokens'] = news_stop_copy['Stop_tokens'].str.replace("[\[\]']", "", regex=True)
news_stop_copy['Stop_tokens'] = news_stop_copy['Stop_tokens'].str.replace(",", "", regex=False)
news_stop_copy

,Date,Title,ppc_Title,Tokens,Stop_tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,"[네이버, 금지, 혐오, 표현, 기준, 개정, 정책, 적용]",금지 혐오 표현 기준 개정 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,"[네이버, 거대, 상공인, 라이브, 커머스, 대본, 작성, 이유]",거대 상공인 라이브 커머스 대본 작성 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,"[네이버, 악성, 댓글, 때, 악플, 꼬리표]",악성 댓글 악플 꼬리표
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,"[네이버, 중소, 상공인, 브랜드, 구축]",중소 상공인 브랜드 구축
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"[네이버, 카카오, 인물, 정보, 장례, 지도사, 직업, 추가]",카카오 인물 정보 장례 지도사 직업 추가
...,...,...,...,...,...
9314,2024-05-27,"티몬, 브랜드 단위 협업으로 직구 강화…""큐텐 시너지로 차별화""",티몬 브랜드 단위 협업으로 직구 강화 큐텐 시너지로 차별화,"[티몬, 브랜드, 단위, 협업, 직구, 강화, 큐, 텐, 시너지, 차별]",티몬 브랜드 단위 협업 직구 강화 큐 텐 시너지 차별
9315,2024-05-27,"카카오·라인야후, 어떻게 해커에 뚫렸나… “오픈채팅방 ID 구조 단순”vs“...",카카오 라인야후 어떻게 해커에 뚫렸나 오픈채팅방 ID 구조 단순 VS,"[카카오, 라인, 라인야후, 해커, 오픈, 채팅방, 구조, 단순]",카카오 라인 라인야후 해커 오픈 채팅방 구조 단순
9316,2024-05-27,네이버웹툰 ‘꿈에서 자유로’ 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실...,네이버웹툰 꿈에서 자유로 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실,"[네이버웹툰, 꿈, 자유, 오디오, 웹툰, 텀, 블럭, 펀딩, 실]",네이버웹툰 꿈 자유 오디오 웹툰 텀 블럭 펀딩 실
9317,2024-05-27,“삼성 출신 들어가니 좀 달라졌네”…요즘 국회는 ‘AI 열공’,삼성 출신 들어가니 좀 달라졌네 요즘 국회는 AI 열공,"[삼성, 출신, 요즘, 국회]",삼성 출신 요즘 국회


In [15]:
# CSV 파일로 저장
news_stop_copy.to_csv('naver_03.Preprocessing_01.csv', index=False)

---

# 9. 결측치 확인 및 제거

In [16]:
import pandas as pd 
news_check = pd.read_csv('naver_03.Preprocessing_01.csv')
news_check


,Date,Title,ppc_Title,Tokens,Stop_tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,"['네이버', '금지', '혐오', '표현', '기준', '개정', '정책', '적용']",금지 혐오 표현 기준 개정 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,"['네이버', '거대', '상공인', '라이브', '커머스', '대본', '작성',...",거대 상공인 라이브 커머스 대본 작성 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,"['네이버', '악성', '댓글', '때', '악플', '꼬리표']",악성 댓글 악플 꼬리표
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,"['네이버', '중소', '상공인', '브랜드', '구축']",중소 상공인 브랜드 구축
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"['네이버', '카카오', '인물', '정보', '장례', '지도사', '직업', ...",카카오 인물 정보 장례 지도사 직업 추가
...,...,...,...,...,...
9314,2024-05-27,"티몬, 브랜드 단위 협업으로 직구 강화…""큐텐 시너지로 차별화""",티몬 브랜드 단위 협업으로 직구 강화 큐텐 시너지로 차별화,"['티몬', '브랜드', '단위', '협업', '직구', '강화', '큐', '텐'...",티몬 브랜드 단위 협업 직구 강화 큐 텐 시너지 차별
9315,2024-05-27,"카카오·라인야후, 어떻게 해커에 뚫렸나… “오픈채팅방 ID 구조 단순”vs“...",카카오 라인야후 어떻게 해커에 뚫렸나 오픈채팅방 ID 구조 단순 VS,"['카카오', '라인', '라인야후', '해커', '오픈', '채팅방', '구조',...",카카오 라인 라인야후 해커 오픈 채팅방 구조 단순
9316,2024-05-27,네이버웹툰 ‘꿈에서 자유로’ 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실...,네이버웹툰 꿈에서 자유로 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실,"['네이버웹툰', '꿈', '자유', '오디오', '웹툰', '텀', '블럭', '...",네이버웹툰 꿈 자유 오디오 웹툰 텀 블럭 펀딩 실
9317,2024-05-27,“삼성 출신 들어가니 좀 달라졌네”…요즘 국회는 ‘AI 열공’,삼성 출신 들어가니 좀 달라졌네 요즘 국회는 AI 열공,"['삼성', '출신', '요즘', '국회']",삼성 출신 요즘 국회


In [17]:
# Tokens 결측치 확인
print(len(news_check[news_check['Stop_tokens'].isna()]))
news_check[news_check['Stop_tokens'].isna()]

9


,Date,Title,ppc_Title,Tokens,Stop_tokens
44,2023-06-02,[잇속] SKT·KT·네이버 외,SKT KT 네이버 외,['네이버'],NaN
3203,2023-09-27,"[10분 뉴스정복] 납작 엎드린 네이버, 시키는대로 한다",납작 엎드린 네이버 시키는대로 한다,['네이버'],NaN
3653,2023-10-20,[잇속] SKT·LGU+·네이버 외,SKT LGU 네이버 외,['네이버'],NaN
4543,2023-11-21,네이버 1784[오후여담],네이버 1784,['네이버'],NaN
5900,2024-01-15,[잇속] LGU+·SKB·네이버 외,LGU SKB 네이버 외,['네이버'],NaN
7175,2024-03-05,[잇속] KT·LGU+·네이버 외,KT LGU 네이버 외,['네이버'],NaN
7493,2024-03-15,[AT 통신·IT 소식] KT·LG U+·네이버,KT LG U 네이버,['네이버'],NaN
7613,2024-03-20,"네이버, 2023년 R&D에 2조원 썼다",네이버 2023년 R D에 2조원 썼다,['네이버'],NaN
9098,2024-05-20,[세상 읽기]네이버는 서두르면 안 된다,네이버는 서두르면 안 된다,['네이버'],NaN


In [18]:
### 5. 결측치 제거
news_check = news_check.dropna(subset=['Stop_tokens'])
news_check[news_check['Stop_tokens'].isna()]

,Date,Title,ppc_Title,Tokens,Stop_tokens


In [19]:
# CSV 파일로 저장
news_check.to_csv('naver_03.Preprocessing_FIN.csv', index=False)

---

In [20]:
# 잘 저장 됐는지 확인
import pandas as pd 
news_check2 = pd.read_csv('naver_03.Preprocessing_FIN.csv')
news_check2

,Date,Title,ppc_Title,Tokens,Stop_tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,"['네이버', '금지', '혐오', '표현', '기준', '개정', '정책', '적용']",금지 혐오 표현 기준 개정 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,"['네이버', '거대', '상공인', '라이브', '커머스', '대본', '작성',...",거대 상공인 라이브 커머스 대본 작성 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,"['네이버', '악성', '댓글', '때', '악플', '꼬리표']",악성 댓글 악플 꼬리표
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,"['네이버', '중소', '상공인', '브랜드', '구축']",중소 상공인 브랜드 구축
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"['네이버', '카카오', '인물', '정보', '장례', '지도사', '직업', ...",카카오 인물 정보 장례 지도사 직업 추가
...,...,...,...,...,...
9305,2024-05-27,"티몬, 브랜드 단위 협업으로 직구 강화…""큐텐 시너지로 차별화""",티몬 브랜드 단위 협업으로 직구 강화 큐텐 시너지로 차별화,"['티몬', '브랜드', '단위', '협업', '직구', '강화', '큐', '텐'...",티몬 브랜드 단위 협업 직구 강화 큐 텐 시너지 차별
9306,2024-05-27,"카카오·라인야후, 어떻게 해커에 뚫렸나… “오픈채팅방 ID 구조 단순”vs“...",카카오 라인야후 어떻게 해커에 뚫렸나 오픈채팅방 ID 구조 단순 VS,"['카카오', '라인', '라인야후', '해커', '오픈', '채팅방', '구조',...",카카오 라인 라인야후 해커 오픈 채팅방 구조 단순
9307,2024-05-27,네이버웹툰 ‘꿈에서 자유로’ 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실...,네이버웹툰 꿈에서 자유로 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실,"['네이버웹툰', '꿈', '자유', '오디오', '웹툰', '텀', '블럭', '...",네이버웹툰 꿈 자유 오디오 웹툰 텀 블럭 펀딩 실
9308,2024-05-27,“삼성 출신 들어가니 좀 달라졌네”…요즘 국회는 ‘AI 열공’,삼성 출신 들어가니 좀 달라졌네 요즘 국회는 AI 열공,"['삼성', '출신', '요즘', '국회']",삼성 출신 요즘 국회


In [21]:
news_check2[news_check2['Stop_tokens'].isna()]

,Date,Title,ppc_Title,Tokens,Stop_tokens


In [24]:
news_check2.iloc[200:250]

,Date,Title,ppc_Title,Tokens,Stop_tokens
200,2023-06-09,"네이버, 자체 보안 기술 ‘NShiftkey’ 솔루션 오픈소스 앱으로 공개",네이버 자체 보안 기술 NSHIFTKEY 솔루션 오픈소스 앱으로 공개,"['네이버', '자체', '보안', '기술', '솔루션', '오픈', '소스', '...",자체 보안 기술 솔루션 오픈 소스 앱 공개
201,2023-06-09,"식약처, 네이버·카카오 등과 온라인 불법행위 자율점검 확대",식약처 네이버 카카오 등과 온라인 불법행위 자율점검 확대,"['식약처', '네이버', '카카오', '온라인', '불법', '행위', '자율',...",식약처 카카오 온라인 불법 행위 자율 점검 확대
202,2023-06-09,"네이버, 챗 GPT 대항마에 힘 싣는다",네이버 챗 GPT 대항마에 힘 싣는다,"['네이버', '대항마', '힘']",대항마 힘
203,2023-06-09,네이버 임직원도 오픈AI CEO에 질문…청중 1000명 몰려,네이버 임직원도 오픈AI CEO에 질문 청중 1000명 몰려,"['네이버', '임직원', '오픈', '질문', '청중']",임직원 오픈 질문 청중
204,2023-06-09,네이버 안에 삼성페이 넣자…월 30회 이상 현장결제하는 이용자 8배 늘었다,네이버 안에 삼성페이 넣자 월 30회 이상 현장결제하는 이용자 8배 늘었다,"['네이버', '안', '삼성', '페이', '월', '이상', '현장', '결제'...",안 삼성 페이 이상 현장 결제 이용자 배
205,2023-06-09,"[클릭 e종목]""네이버, 경기불황에도 실적 순항""",네이버 경기불황에도 실적 순항,"['네이버', '경기', '불황', '실적', '순항']",경기 불황 실적 순항
206,2023-06-09,쿠팡은 배송·네이버는 적립… 신세계 멤버십 혜택은?,쿠팡은 배송 네이버는 적립 신세계 멤버십 혜택은,"['쿠팡', '배송', '네이버', '적립', '신세계', '멤버십', '혜택']",쿠팡 배송 적립 신세계 멤버십 혜택
207,2023-06-09,네이버·카카오 등 국내 부가통신사업자 작년 876조 매출 올려,네이버 카카오 등 국내 부가통신사업자 작년 876조 매출 올려,"['네이버', '카카오', '국내', '부가', '통신', '사업자', '작년', ...",카카오 국내 부가 통신 사업자 작년 매출
208,2023-06-09,"네이버·카카오, 포털 댓글부터 뉴스까지 싹 바꾼다",네이버 카카오 포털 댓글부터 뉴스까지 싹 바꾼다,"['네이버', '카카오', '포털', '댓글', '뉴스']",카카오 포털 댓글 뉴스
209,2023-06-09,"[유통가 레이더] '최대 53% 파격 할인'… 남양유업, '네이버 도착보장 메가세...",최대 53 파격 할인 남양유업 네이버 도착보장 메가세,"['최대', '파격', '할인', '남양유업', '네이버', '도착', '보장']",최대 파격 할인 남양유업 도착 보장


In [22]:
news_check2[news_check2['Stop_tokens'].apply(lambda x: '카카오' in x)].head(50)

,Date,Title,ppc_Title,Tokens,Stop_tokens
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"['네이버', '카카오', '인물', '정보', '장례', '지도사', '직업', ...",카카오 인물 정보 장례 지도사 직업 추가
7,2023-06-01,"AI 웹툰 논란에 네이버·카카오 ""인간 작품만 받는다""",AI 웹툰 논란에 네이버 카카오 인간 작품만 받는다,"['웹툰', '논란', '네이버', '카카오', '인간', '작품']",웹툰 논란 카카오 인간 작품
26,2023-06-01,"서비스 개편 나선 네이버ㆍ카카오, 새 먹거리 발굴 속도",서비스 개편 나선 네이버ㆍ카카오 새 먹거리 발굴 속도,"['서비스', '개편', '네이버', '카카오', '먹거리', '발굴', '속']",서비스 개편 카카오 먹거리 발굴 속
51,2023-06-02,"‘장례지도사’ ‘웨딩플래너’ 등 5개 직업군, 네이버·카카오 인물정보에 등...",장례지도사 웨딩플래너 등 5개 직업군 네이버 카카오 인물정보에 등,"['장례', '지도사', '웨딩플래너', '직업군', '네이버', '카카오', '인...",장례 지도사 웨딩플래너 직업군 카카오 인물 정보
52,2023-06-02,"네이버, 핀테크 관련 상장기업 브랜드평판 1위···카카오페이·KT 뒤쫓아",네이버 핀테크 관련 상장기업 브랜드평판 1위 카카오페이 KT 뒤쫓아,"['네이버', '핀테크', '관련', '상장', '기업', '브랜드', '평판', ...",핀테크 관련 상장 기업 브랜드 평판 카카오페이
56,2023-06-02,"네이버·카카오, '성소수자(LGBTQ) 혐오' 표현 제재 나서",네이버 카카오 성소수자 LGBTQ 혐오 표현 제재 나서,"['네이버', '카카오', '성', '소수자', '혐오', '표현', '제재']",카카오 소수자 혐오 표현 제재
79,2023-06-05,치솟는 구글 점유율에 흔들리는 네이버·카카오,치솟는 구글 점유율에 흔들리는 네이버 카카오,"['구글', '점유', '네이버', '카카오']",구글 점유 카카오
82,2023-06-05,"""나 떨고 있니?"" 카카오톡, 네이버 제친 유튜브에 추월 위기",나 떨고 있니 카카오톡 네이버 제친 유튜브에 추월 위기,"['카카오', '네이버', '유튜브', '추월', '위기']",카카오 유튜브 추월 위기
83,2023-06-05,[朝鮮칼럼] 문제는 뉴스제평위가 아니라 네이버·카카오다,문제는 뉴스제평위가 아니라 네이버 카카오다,"['문제', '뉴스제평위', '네이버', '카카오']",문제 뉴스제평위 카카오
85,2023-06-05,"네이버·카카오 ""종이→전자문서로 디지털 친환경 앞장서""(종합)",네이버 카카오 종이 전자문서로 디지털 친환경 앞장서 종합,"['네이버', '카카오', '종이', '전자', '문서', '디지털', '환경', ...",카카오 종이 전자 문서 디지털 환경 종합


In [23]:
news_check2.iloc[120:200]

,Date,Title,ppc_Title,Tokens,Stop_tokens
120,2023-06-07,"현대오토에버, 네이버 출신 정수환 상무 영입",현대오토에버 네이버 출신 정수환 상무 영입,"['현대', '오토', '에버', '네이버', '출신', '정수화', '상무', '...",현대 오토 에버 출신 정수화 상무 영입
121,2023-06-07,"'네이버 투자'받은 퓨리오사AI, LG '엑사원' 생성형 AI 검증 어찌 하나",네이버 투자 받은 퓨리오사AI LG 엑사원 생성형 AI 검증 어찌 하나,"['네이버', '투자', '퓨리오사', '액', '사원', '생성', '검증']",투자 퓨리오사 액 사원 생성 검증
122,2023-06-07,"“네이버, 강점은 커머스 분야…플랫폼 최선호株”",네이버 강점은 커머스 분야 플랫폼 최선호株,"['네이버', '강점', '커머스', '분야', '플랫폼', '선호']",강점 커머스 분야 플랫폼 선호
123,2023-06-07,"현대오토에버, 클라우드 강화…네이버 출신 정수환 상무 영입",현대오토에버 클라우드 강화 네이버 출신 정수환 상무 영입,"['현대', '오토', '에버', '클라우드', '강화', '네이버', '출신', ...",현대 오토 에버 클라우드 강화 출신 정수화 상무 영입
124,2023-06-07,"aT, 네이버 쇼핑 라이브에서 ‘플럼코트’ 등 신품종 선보여",AT 네이버 쇼핑 라이브에서 플럼코트 등 신품종 선보여,"['네이버', '쇼핑', '라이브', '풀럼', '코트', '품종']",쇼핑 라이브 풀럼 코트 품종
...,...,...,...,...,...
195,2023-06-08,美 금리인상 우려 재차 부각…네이버·카카오 동반 약세,美 금리인상 우려 재차 부각 네이버 카카오 동반 약세,"['금리', '인상', '우려', '부각', '네이버', '카카오', '동반', '...",금리 인상 우려 부각 카카오 동반 약세
196,2023-06-08,"eoe의 고식이섬유 보충제 이너딜리티, '네이버 원쁠딜' 행사 성료",EOE의 고식이섬유 보충제 이너딜리티 네이버 원쁠딜 행사 성료,"['식이', '섬유', '보충', '이너딜리티', '네이버', '원쁠딜', '행사'...",식이 섬유 보충 이너딜리티 원쁠딜 행사 료
197,2023-06-09,"악플 사라질까…네이버 '악플러 노출' ,다음 '하루살이 채팅'",악플 사라질까 네이버 악플러 노출 다음 하루살이 채팅,"['악플', '네이버', '악플', '노출', '다음', '하루살이', '채팅']",악플 악플 노출 다음 하루살이 채팅
198,2023-06-09,"네이버웹툰 원작 '사냥개들', 넷플릭스 공개",네이버웹툰 원작 사냥개들 넷플릭스 공개,"['네이버웹툰', '원작', '사냥개', '넷플릭스', '공개']",네이버웹툰 원작 사냥개 넷플릭스 공개
